## EDA

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('drive/My Drive/Sirius AI/transaction_dataset.csv',).drop(columns=['Unnamed: 0'])

In [ ]:
#df = pd.read_csv(transaction_dataset.csv',).drop(columns=['Unnamed: 0'])

In [ ]:
assert df.shape[0] == df.drop_duplicates().shape[0]
df = df.drop(columns='Index').drop_duplicates()
len(df)

9823

In [ ]:
df = df[df.columns[(df.nunique() != 1)]]
len(df.columns)

42

In [ ]:
feature_list = list(set(df.columns) - set(['Address', 'FLAG']))

num_feature_list = list(set(feature_list) - set(df.dtypes[df.dtypes == 'object'].index))
cat_feature_list = list(set(feature_list) - set(num_feature_list))

In [ ]:
df[num_feature_list + ['FLAG']].corr()[['FLAG']]

,FLAG
Number of Created Contracts,-0.013741
min val sent,0.006603
ERC20 uniq rec contract addr,-0.018509
ERC20 total Ether received,-0.004486
ERC20 max val sent,0.025034
ERC20 uniq rec token name,-0.018029
ERC20 uniq sent token name,0.009490
ERC20 min val sent,0.025359
total ether sent contracts,-0.008005
Total ERC20 tnxs,-0.025737


In [ ]:
pairs = df[num_feature_list].corr().stack()
pairs_list = list(pairs[pairs.abs().gt(0.9) & pairs.abs().lt(1.0)].index)
uniq_pairs_list = []

for x, y in pairs_list:
    if (x, y) in uniq_pairs_list or (y, x) in uniq_pairs_list:
        continue
    else:
        uniq_pairs_list.append((x, y))

filter_cor_feature_list =  list({x[0] for x in uniq_pairs_list})

In [ ]:
#df = df[list((set(num_feature_list) - set(filter_cor_feature_list)) | set(['FLAG'])) + cat_feature_list]

In [ ]:
df1 = df[list((set(num_feature_list) - set(filter_cor_feature_list))) + ['FLAG']].corr()[['FLAG']].sort_values(by = ['FLAG'])
features = list(df1[abs(df1['FLAG']) >= 0.02].index)

In [ ]:
df = df[features]

In [ ]:
for ind in features:
    med = df[ind].median()
    Q1, Q3 = df[ind].quantile([0.25, 0.75])
    IQR = Q3 - Q1

    down_bord, up_bord = med - 1.5 * IQR, med + 1.5 * IQR
    if ind != 'FLAG':
      data = df[(df[ind] >= down_bord) & (df[ind] <= up_bord)]
len(data)

7530

In [ ]:
for ind in features:
    data[ind] = data[ind]. fillna (data[ind].median())

In [ ]:
data

,Time Diff between first and last (Mins),Avg min between received tnx,total transactions (including tnx to create contract,Received Tnx,Sent tnx,avg val sent,Unique Sent To Addresses,Unique Received From Addresses,Avg min between sent tnx,Total ERC20 tnxs,max val sent,min value received,ERC20 uniq sent addr,ERC20 avg val sent,FLAG
2,516729.30,2434.02,12,10,2,1.794308,2,10,246194.54,8.0,3.538616,0.113119,0.0,0.0,0
5,20926.68,375.48,5,3,2,4.851858,1,2,9900.12,0.0,5.536916,0.724148,0.0,0.0,0
6,8660.35,629.44,36,11,25,0.482496,20,9,69.46,0.0,4.284785,0.049000,0.0,0.0,0
7,319828.05,176.84,218,5,213,0.040861,3,3,1497.39,7.0,0.700000,0.118490,0.0,0.0,0
8,496.62,0.00,2,1,1,1.999380,1,1,0.00,0.0,1.999380,2.000000,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9832,0.00,0.00,0,0,0,0.000000,0,0,0.00,2.0,0.000000,0.000000,0.0,0.0,1
9836,58748.48,631.39,17,13,4,9.166365,4,11,12635.10,2.0,12.200000,0.004082,0.0,0.0,1
9837,0.00,0.00,0,0,0,0.000000,0,0,0.00,1.0,0.000000,0.000000,0.0,0.0,1
9838,261601.88,2189.29,110,43,67,0.922179,44,31,2499.44,5.0,9.500000,0.001078,0.0,0.0,1


In [ ]:
data['FLAG'].nunique()

2

## ML

In [ ]:
from sklearn.model_selection import train_test_split

In [31]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(['FLAG'], axis=1), data['FLAG'],
                                                    test_size = 0.1, random_state = 42, stratify=data['FLAG'])

In [32]:
y_test.nunique()

2

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [35]:
clf = RandomForestClassifier(random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [36]:
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, clf.predict(X_test))))

Model accuracy score: 0.9854


In [37]:
feature_scores = pd.Series(clf.feature_importances_, index=X_train.columns).sort_values(ascending=False)

feature_scores

 Total ERC20 tnxs                                       0.236550
Time Diff between first and last (Mins)                 0.140304
Unique Received From Addresses                          0.113139
total transactions (including tnx to create contract    0.107361
Avg min between received tnx                            0.095275
Received Tnx                                            0.087123
min value received                                      0.053182
avg val sent                                            0.039400
max val sent                                            0.037555
Sent tnx                                                0.036875
Avg min between sent tnx                                0.034803
Unique Sent To Addresses                                0.013478
 ERC20 uniq sent addr                                   0.004954
 ERC20 avg val sent                                     0.000000
dtype: float64

# Final model

In [ ]:
X_train.columns

Index(['Time Diff between first and last (Mins)',
       'Avg min between received tnx',
       'total transactions (including tnx to create contract', 'Received Tnx',
       'Sent tnx', 'avg val sent', 'Unique Sent To Addresses',
       'Unique Received From Addresses', 'Avg min between sent tnx',
       ' Total ERC20 tnxs', 'max val sent', 'min value received',
       ' ERC20 uniq sent addr', ' ERC20 avg val sent'],
      dtype='object')

In [38]:
data.columns

Index(['Time Diff between first and last (Mins)',
       'Avg min between received tnx',
       'total transactions (including tnx to create contract', 'Received Tnx',
       'Sent tnx', 'avg val sent', 'Unique Sent To Addresses',
       'Unique Received From Addresses', 'Avg min between sent tnx',
       ' Total ERC20 tnxs', 'max val sent', 'min value received',
       ' ERC20 uniq sent addr', ' ERC20 avg val sent', 'FLAG'],
      dtype='object')

In [39]:
feature_drop = ['Unique Sent To Addresses', ' ERC20 uniq sent addr', ' ERC20 avg val sent', 'FLAG'].copy()
X_train, X_test, y_train, y_test = train_test_split(data.drop(feature_drop, axis=1), data['FLAG'],
                                                    test_size = 0.1, random_state = 42, stratify=data['FLAG'])

In [40]:
rfc = RandomForestClassifier(n_estimators= 100, random_state= 0)
rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [41]:
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, rfc.predict(X_test))))

Model accuracy score: 0.9867


In [42]:
feature_scores = pd.Series(rfc.feature_importances_, index=X_train.columns).sort_values(ascending=False)

feature_scores

 Total ERC20 tnxs                                       0.263329
Time Diff between first and last (Mins)                 0.189419
Unique Received From Addresses                          0.109395
total transactions (including tnx to create contract    0.105397
Avg min between received tnx                            0.087563
Received Tnx                                            0.061904
min value received                                      0.056577
max val sent                                            0.035454
avg val sent                                            0.032973
Sent tnx                                                0.030873
Avg min between sent tnx                                0.027114
dtype: float64

In [43]:
print(*X_train.columns, sep= '\n')

Time Diff between first and last (Mins)
Avg min between received tnx
total transactions (including tnx to create contract
Received Tnx
Sent tnx
avg val sent
Unique Received From Addresses
Avg min between sent tnx
 Total ERC20 tnxs
max val sent
min value received


# Работа с реальными данными

In [47]:
import requests
import datetime

# Некоторая информация о транзакции

# Замените YOUR_API_KEY_HERE на ваш API ключ
API_KEY = 'BWAD75WZB8EU8RQX2X8C9J84WXP7RCQJXZ'
DDRESS = '0x4838B106FCe9647Bdf1E7877BF73cE8B0BAD5f97'

def get_transactions(address):
    url = f'https://api.etherscan.io/api?module=account&action=txlist&address={address}&startblock=0&endblock=99999999&sort=asc&apikey={API_KEY}'
    response = requests.get(url)
    transactions = response.json()['result']
    return transactions

def analyze_transactions(transactions):
    if not transactions:
        return "Нет транзакций для анализа."

    # Время между первой и последней транзакцией
    first_transaction_time = datetime.datetime.utcfromtimestamp(int(transactions[0]['timeStamp']))
    last_transaction_time = datetime.datetime.utcfromtimestamp(int(transactions[-1]['timeStamp']))
    time_between_first_last = (last_transaction_time - first_transaction_time).total_seconds() / 60
    #time_between_first_last = (last_transaction_time - first_transaction_time).timestamp() / (60 * 10**6)

    # Общее количество транзакций
    total_transactions = len(transactions)

    # Дополнительная аналитика здесь...
    # Например, вы можете итерировать через все транзакции для расчета средних значений,
    # подсчета уникальных адресов и т.д.

    return {
        'ADDRESS': ADDRESS,
        #'first_transaction_time': first_transaction_time,
        #'last_transaction_time': last_transaction_time,
        'time_between_first_last': time_between_first_last,
        'total_transactions': total_transactions,
        # Добавьте другие вычисленные статистики сюда...
    }

# Получаем транзакции
transactions = get_transactions(ADDRESS)

# Анализируем транзакции
analysis = analyze_transactions(transactions)
print(analysis)

{'ADDRESS': '0x4838B106FCe9647Bdf1E7877BF73cE8B0BAD5f97', 'time_between_first_last': 137824.0, 'total_transactions': 10000}


In [48]:
import pandas as pd
import requests

def run_query(uri, query):
    request = requests.post(uri, json={'query': query}, headers={"Content-Type": "application/json"})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception(f"Unexpected status code returned: {request.status_code}")

url = "https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2"
query = """{
  markets(first: 7) {
    accrualBlockNumber
    borrowRate
    cash
    collateralFactor
    exchangeRate
    interestRateModelAddress
    name
    reserves
    supplyRate
    symbol
    id
    totalBorrows
    totalSupply
    underlyingAddress
    underlyingName
    underlyingPrice
    underlyingSymbol
    reserveFactor
    underlyingPriceUSD
  }
}"""

# Подробная информация о транзакции
result = run_query(url, query)